In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.patches import Polygon
from scipy.spatial import ConvexHull

from avstack.geometry import (
    q_cam_to_stan,
    q_stan_to_cam,
    q_mult_vec,
    transformations as tforms,
)
from avstack.maskfilters import filter_points_in_image

In [ ]:
agent_colors = list(mcolors.TABLEAU_COLORS.keys())
track_colors = list(mcolors.XKCD_COLORS.keys())


def get_track_color(ID_track):
    return track_colors[ID_track % len(track_colors)]


def get_agent_color(ID_agent):
    return agent_colors[ID_agent % len(agent_colors)]

In [ ]:
n_cameras = 4

# ----------------------------------------
# EXTRINSICS -- from 3D world to 3D camera
# ----------------------------------------

d2r = np.pi / 180
pitch = -30

# rotation
cam_rotations = {
    0: q_cam_to_stan
    * tforms.transform_orientation([0, pitch * d2r, 45 * d2r], "euler", "quat"),
    1: q_cam_to_stan
    * tforms.transform_orientation([0, pitch * d2r, -45 * d2r], "euler", "quat"),
    2: q_cam_to_stan
    * tforms.transform_orientation([0, pitch * d2r, -135 * d2r], "euler", "quat"),
    3: q_cam_to_stan
    * tforms.transform_orientation([0, pitch * d2r, 135 * d2r], "euler", "quat"),
}

# position -- specified in world frame, converted to sensor frame
cam_positions = {
    0: np.array([0, 0, 4]),
    1: np.array([0, 5, 4]),
    2: np.array([5, 5, 4]),
    3: np.array([5, 0, 4]),
}

# full transforms
cam_extrinsics = {
    i: np.block(
        [
            [
                tforms.transform_orientation(
                    cam_rotations[i].conjugate(), "quat", "dcm", n_prec=6
                ),
                np.reshape(
                    q_mult_vec(
                        cam_rotations[i].conjugate(), -cam_positions[i], n_prec=6
                    ),
                    (3, 1),
                ),
            ],
            [np.zeros((1, 3)), np.ones((1, 1))],
        ]
    )
    for i in range(n_cameras)
}

# ----------------------------------------
# INTRINSICS - from 3D camera to 2D image
# ----------------------------------------
width = 2448
height = 2048
f = 3.5 * 1e-3
mx = width / (8.80 * 1e-3)
my = height / (6.60 * 1e-3)
P_cam = np.array(
    [
        [f * mx, 0, width / 2, 0],
        [0, f * my, height / 2, 0],
        [0, 0, 1, 0],
    ],
    dtype=float,
)
cam_intrinsics = {i: P_cam for i in range(n_cameras)}
img_size = {i: (width, height) for i in range(n_cameras)}

In [ ]:
# Create the discretized grid of points in BEV
extent = [[-0.5, 6], [-0.5, 6]]
dx = 0.1
X, Y = np.meshgrid(*[np.arange(extent[dim][0], extent[dim][1], dx) for dim in range(2)])
Z = np.zeros(X.size)
pt_bev_tuples = np.vstack([X.ravel(), Y.ravel(), Z])
pt_bev_tuples_homog = np.vstack([pt_bev_tuples, np.ones(pt_bev_tuples.shape[1])])

In [ ]:
# Check the point visibility
boundary = {}
points_visible = {}
for cam in range(1):
    # project into camera image plane
    pts_in_cam = cam_extrinsics[cam] @ pt_bev_tuples_homog
    points_in_img = tforms.project_to_image(pts_in_cam[:3, :].T, cam_intrinsics[cam])
    visible_this_cam = filter_points_in_image(pts_in_cam[:3, :].T, cam_intrinsics[cam])

    # consistency check
    if sum(visible_this_cam) == 0:
        raise RuntimeError(f"No points found in FOV for camera {cam}")

    # perform convex hull estimation for boundary
    points_visible[cam] = pt_bev_tuples[:3, visible_this_cam]
    hull = ConvexHull(pt_bev_tuples[:2, visible_this_cam].T)
    boundary[cam] = np.array(
        [pt_bev_tuples[:, visible_this_cam][:2, v] for v in hull.vertices]
    )

In [ ]:
# Plot the result
fig, ax = plt.subplots()
for cam, pos in cam_positions.items():
    # camera positions
    ax.scatter(
        pos[0], pos[1], marker="x", label=f"Camera {cam}", color=get_agent_color(cam)
    )
    # camera pointing angle
    length = 0.25
    R = tforms.transform_orientation(q_stan_to_cam * cam_rotations[cam])
    dx = 
    dy = 
    ax.arrow(x=pos[0], y=pos[1], dx=dx, dy=dy, length_includes_head=True,
          head_width=0.08, head_length=0.00002)
    # camera fovs
    p = Polygon(boundary[cam], facecolor=get_agent_color(cam), alpha=0.1)
    ax.add_patch(p)
    break
ax.set_xlim(extent[0])
ax.set_ylim(extent[1])
plt.legend()
plt.show()